In [1]:
#1.3 ellipsoid
# Let M be an ellipsoid in R3, (ax)^{2}+(by)^{2}+z^{2}=d
# x = 1/a*cos(p)*sin(t)*e1 + 1/b*sin(p)*sin(t)*e2 + cos(t)*e3, t, p in R

from gc_utils import *
alg = Algebra(3)
locals().update(alg.blades)

c2e = lambda p, t: 4*np.cos(p)*np.sin(t)*e1 + 5*np.sin(p)*np.sin(t)*e2 + np.cos(t)*e3
e_p = lambda p, t: -4*np.sin(p)*np.sin(t)*e1 + 5*np.cos(p)*np.sin(t)*e2
e_t = lambda p, t: (4*np.cos(p)*np.cos(t)*e1) + (5*np.sin(p)*np.cos(t)*e2) - (np.sin(t)*e3)
def Ix(x):
    p = np.arctan((4/5)*x.e2/x.e1)
    t = np.arccos(x.e3)
    return e_p(p, t) ^ e_t(p, t)

# |p| < pi/2, 0 < t < pi for unique arctan & arccos
p, t = -1, 3
x = c2e(p, t)
I = Ix(x)
nIx = lambda x: normalize(Ix(x))
nI = nIx(x)

nI, nI ^ (3*e_t(p,t) + 5*e_p(p,t))

(1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃, 1.29e-15 𝐞₁₂₃)

In [2]:
#5.1
C, D, E = [random_multivector(alg) for _ in range(3)]
V1, V2 = [random_versor(2, alg) for _ in range(2)]
f = lambda x: V1.sw(inv(x)) + V2.sw(x**3)
A = lambda x: E*x**2 + C*x*D
a = lambda x: A(x).grade(1)
PA = lambda x: P(A(x), Ix(x))
pa = lambda x: P(a(x), Ix(x))
f_ = lambda a: lambda x: differential(f, x, a)

x, f(x), pa(x), f_(pa(x))(x)

(0.305 𝐞₁ + -0.594 𝐞₂ + -0.99 𝐞₃,
 2.11 𝐞₁ + -1.77 𝐞₂ + 0.0853 𝐞₃,
 0.57 𝐞₁ + 0.584 𝐞₂ + -0.00304 𝐞₃,
 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃)

In [3]:
#5.2 For tangent fields, the outermorphism can be computed in the embedded space
# the scalar product in the skew_symmetrizer acts as a projection into the tangent space
def outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        A = field(x)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

outermorphism(f_, A, alg)(x), outermorphism(f_, A, alg, Ix=Ix)(x), outermorphism(f_, PA, alg)(x), outermorphism(f_, PA, alg, Ix=Ix)(x)

(0.603 + 0.257 𝐞₁ + 0.859 𝐞₂ + -1.47 𝐞₃ + 0.41 𝐞₁₂ + -5.78 𝐞₁₃ + 6.02 𝐞₂₃ + 7.59 𝐞₁₂₃,
 0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃ + 4.06e-18 𝐞₁₂₃,
 0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃,
 0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃ + 4.06e-18 𝐞₁₂₃)

In [4]:
#5.3
f_vec = lambda a: lambda x: differential(f, x, a)
f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
I1 = f_(Ix)(x)
nI1 = f_(nIx)(x)
norm(I1)/norm(I), norm(nI1)

(6.024958532921703, 6.024958532921705)

In [5]:
# Jacobian
Jf = lambda x: norm(f_(nIx)(x))

In [6]:
#5.4
def adjoint_outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        A = field(x)
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

def adjoint_outermorphism_(f, A, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

adjoint_outermorphism(f_vec, A, alg, Ix=Ix)(x)

0.603 + 1.95 𝐞₁ + -0.684 𝐞₂ + 0.054 𝐞₃ + -5.64 𝐞₁₂ + 0.135 𝐞₁₃ + 0.109 𝐞₂₃

In [7]:
240/(2*10*200/1000*7.3), 100/ (np.pi*49/4), 130/100*5

(8.219178082191782, 2.598448050479924, 6.5)

In [8]:
#5.10a
f_(A)(x), f_(PA)(x), P(f_(A)(x), I1)

(0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃ + 4.06e-18 𝐞₁₂₃,
 0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃ + 4.06e-18 𝐞₁₂₃,
 0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃)

In [9]:
#5.10b
P_ = lambda A: lambda x: P(A(x), I)
P__ = lambda A: lambda x: P(A(x), I1)
fa = lambda A: adjoint_outermorphism(f_vec, A, alg, Ix=Ix)
fa(A)(x), fa(P__(A))(x), P_(fa(A))(x)

(0.603 + 1.95 𝐞₁ + -0.684 𝐞₂ + 0.054 𝐞₃ + -5.64 𝐞₁₂ + 0.135 𝐞₁₃ + 0.109 𝐞₂₃,
 0.603 + 1.95 𝐞₁ + -0.684 𝐞₂ + 0.054 𝐞₃ + -5.64 𝐞₁₂ + 0.135 𝐞₁₃ + 0.109 𝐞₂₃,
 0.603 + 1.95 𝐞₁ + -0.684 𝐞₂ + 0.054 𝐞₃ + -5.64 𝐞₁₂ + 0.135 𝐞₁₃ + 0.109 𝐞₂₃)

In [10]:
#5.12 outermorphism push forward tangent vectors
G = lambda x: (x+D)**2
F = lambda x: G(f(x))
differential(F, x, pa(x)), differential(G, f(x), f_(pa)(x))

(-3.16 + 0.119 𝐞₁ + 0.807 𝐞₂ + -1.19 𝐞₃ + -2.54 𝐞₁₂ + -1.72 𝐞₁₃ + 0.252 𝐞₂₃ + -3.17 𝐞₁₂₃,
 -3.17 + 0.119 𝐞₁ + 0.807 𝐞₂ + -1.19 𝐞₃ + -2.54 𝐞₁₂ + -1.72 𝐞₁₃ + 0.252 𝐞₂₃ + -3.17 𝐞₁₂₃)

In [43]:
#5.12c adjoint-outermorphism is the pullback/chain rule of derivatives
# without explicit define the inverse of f, we push forward the frame

def f_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    for v, r in frame_gen(x, alg, f_(Ix)):
        yield (r, lambda F: differential(F, f(x), v, h))

# Say f: M -> N
# "back" as the derivative acts on a field G on N
# Yet mimicking the derivative of G(f(x)) on M
def back_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix)
    for v, r in frame_gen(x, alg, f_(Ix)):
        yield (fa_(r)(x), lambda F: differential(F, f(x), v, h))


fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix)
differential(F, x, pa(x)), sum((pa(x)|r) * o(G) for r, o in back_derivative_gen(x, f, alg, Ix))

(-3.16 + 0.119 𝐞₁ + 0.807 𝐞₂ + -1.19 𝐞₃ + -2.54 𝐞₁₂ + -1.72 𝐞₁₃ + 0.252 𝐞₂₃ + -3.17 𝐞₁₂₃,
 -3.17 + 0.119 𝐞₁ + 0.807 𝐞₂ + -1.19 𝐞₃ + -2.54 𝐞₁₂ + -1.72 𝐞₁₃ + 0.252 𝐞₂₃ + -3.17 𝐞₁₂₃)

In [45]:
#5.13
(
    derivative(F, x, alg, Ix=Ix), 
    sum(r * o(G) for r, o in back_derivative_gen(x, f, alg, Ix))
    )

(2.17 + 8.82 𝐞₁ + -12.4 𝐞₂ + -5.12 𝐞₃ + 2.68 𝐞₁₂ + 3.92 𝐞₁₃ + -2.33 𝐞₂₃ + 4.62 𝐞₁₂₃,
 2.17 + 8.82 𝐞₁ + -12.4 𝐞₂ + -5.12 𝐞₃ + 2.68 𝐞₁₂ + 3.92 𝐞₁₃ + -2.34 𝐞₂₃ + 4.62 𝐞₁₂₃)

In [13]:
def outermorphism_(f, A, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

In [42]:
#5.14 "forward" as the derivative acts on a field F on M
# Yet mimicking the derivative of F(inv(f)(x)) on N
def forward_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    nIx = lambda x: normalize(Ix(x))
    nI = nIx(x)
    iI1 = inv(f_(nIx)(x))
    for v, r in frame_gen(x, alg, Ix):
        yield iI1 * f__(nI*r)(x), lambda F: differential(F, x, v, h)

iI1 = inv(f_(nIx)(x))
f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix)
(
    sum(r * o(G) for r, o in f_derivative_gen(x, f, alg, Ix)),
    sum(r * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix)),
    sum(iI1 * f__(nI*r)(x) * o(F) for r, o in derivative_gen(x, alg, Ix))
    )

(1.71 + 3.67 𝐞₁ + -0.218 𝐞₂ + 2.48 𝐞₃ + 1.75 𝐞₁₂ + 0.866 𝐞₁₃ + 0.679 𝐞₂₃ + 3.63 𝐞₁₂₃,
 1.71 + 3.67 𝐞₁ + -0.218 𝐞₂ + 2.48 𝐞₃ + 1.75 𝐞₁₂ + 0.866 𝐞₁₃ + 0.679 𝐞₂₃ + 3.63 𝐞₁₂₃,
 1.71 + 3.67 𝐞₁ + -0.218 𝐞₂ + 2.48 𝐞₃ + 1.75 𝐞₁₂ + 0.866 𝐞₁₃ + 0.679 𝐞₂₃ + 3.63 𝐞₁₂₃)

In [15]:
#5.15a Without the inverse of f
# Pushing forward a differential is an extra linear transformation
b = random_r_blade(1, alg)
pb = P(b, I)
pb1 = f__(pb)(x)
differential(f__(PA(x)), x, pb, h=1e-2), sum((pb1 | r) * o(f__(PA(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2))

(0.623 𝐞₁ + -0.319 𝐞₂ + -0.224 𝐞₃ + 0.443 𝐞₁₂ + 0.122 𝐞₁₃ + 0.243 𝐞₂₃ + -2.04e-16 𝐞₁₂₃,
 -4.71e-17 + 0.632 𝐞₁ + -0.304 𝐞₂ + -0.225 𝐞₃ + 0.427 𝐞₁₂ + 0.135 𝐞₁₃ + 0.249 𝐞₂₃ + 9.87e-16 𝐞₁₂₃)

In [16]:
#5.15b
differential(fa_(PA(x)), x, pb, h=1e-2), sum((pb1 | r) * o(fa_(PA(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2))

(0.352 𝐞₁ + -0.0536 𝐞₂ + 0.0409 𝐞₃ + 0.435 𝐞₁₂ + -0.0252 𝐞₁₃ + 0.00926 𝐞₂₃,
 -5.69e-17 + 0.377 𝐞₁ + -0.0541 𝐞₂ + 0.0414 𝐞₃ + 0.419 𝐞₁₂ + -0.0248 𝐞₁₃ + 0.00957 𝐞₂₃ + -4.21e-18 𝐞₁₂₃)

In [17]:
#5.16
(
    differential(f__(C^D), x, pb, h=1e-3), 
    (differential(f__(C), x, pb, h=1e-3)^f__(D)(x)) + (f__(C)(x)^differential(f__(D), x, pb, h=1e-3)),
    differential(fa_(C^D), x, pb, h=1e-3), 
    (differential(fa_(C), x, pb, h=1e-3)^fa_(D)(x)) + (fa_(C)(x)^differential(fa_(D), x, pb, h=1e-3)),
    )

(0.114 𝐞₁ + -0.098 𝐞₂ + -0.0229 𝐞₃ + 0.029 𝐞₁₂ + 0.0094 𝐞₁₃ + 0.0162 𝐞₂₃ + -2.21e-16 𝐞₁₂₃,
 0.114 𝐞₁ + -0.098 𝐞₂ + -0.0229 𝐞₃ + 0.029 𝐞₁₂ + 0.00941 𝐞₁₃ + 0.0162 𝐞₂₃ + 2.74e-05 𝐞₁₂₃,
 0.00715 𝐞₁ + -0.1 𝐞₂ + 0.003 𝐞₃ + -0.274 𝐞₁₂ + -0.0219 𝐞₁₃ + 0.0393 𝐞₂₃,
 0.00715 𝐞₁ + -0.1 𝐞₂ + 0.003 𝐞₃ + -0.274 𝐞₁₂ + -0.0219 𝐞₁₃ + 0.0393 𝐞₂₃ + 4.54e-06 𝐞₁₂₃)

In [18]:
#5.17a
Pb_ = lambda A: lambda x: differential(lambda a: P(A(x), Ix(a)), x, pb, h=1e-2)
Pb_1 = lambda A: lambda x: differential(lambda a: P(A(x), f_(Ix)(a)), x, pb, h=1e-2)
fb = lambda C: differential(f__(C), x, pb, h=1e-2)
C_ = lambda x: C
(
    fb(C),
    Pb_1(f__(C))(x)+ P(fb(C), I1),
    fb(P(C, I)) + f_(Pb_(C_))(x)
    )

(0.242 𝐞₁ + -0.248 𝐞₂ + -0.0312 𝐞₃ + -0.155 𝐞₁₂ + -0.0578 𝐞₁₃ + -0.0944 𝐞₂₃ + -1.01e-16 𝐞₁₂₃,
 -6.94e-16 + 0.242 𝐞₁ + -0.248 𝐞₂ + -0.0311 𝐞₃ + -0.155 𝐞₁₂ + -0.0578 𝐞₁₃ + -0.0944 𝐞₂₃,
 0.242 𝐞₁ + -0.248 𝐞₂ + -0.0312 𝐞₃ + -0.155 𝐞₁₂ + -0.0578 𝐞₁₃ + -0.0944 𝐞₂₃ + -1.01e-16 𝐞₁₂₃)

In [19]:
#5.17b
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)

(
    fb(C),
    Pb_(fa_(C))(x)+ P(fb(C), I),
    fb(P(C, I1)) + fa_(Pb_1(C_)(x))(x)
    )

(0.0391 𝐞₁ + -0.273 𝐞₂ + 0.00836 𝐞₃ + -0.83 𝐞₁₂ + -0.0361 𝐞₁₃ + 0.0829 𝐞₂₃,
 0.0391 𝐞₁ + -0.273 𝐞₂ + 0.00837 𝐞₃ + -0.83 𝐞₁₂ + -0.0361 𝐞₁₃ + 0.0829 𝐞₂₃,
 -6.94e-16 + 0.0391 𝐞₁ + -0.273 𝐞₂ + 0.00837 𝐞₃ + -0.83 𝐞₁₂ + -0.0361 𝐞₁₃ + 0.0829 𝐞₂₃)

In [20]:
#5.18
differential(lambda y: differential(f, y, a(x)), x, b), differential(lambda y: differential(f, y, b), x, a(x))

(0.742 𝐞₁ + -0.963 𝐞₂ + -0.29 𝐞₃, 0.742 𝐞₁ + -0.963 𝐞₂ + -0.29 𝐞₃)

In [21]:
#5.19
tangents = blade_split(I, alg)
frame = multi_frame(tangents)
r_frame = reci_frame(tangents)
# pb_frame = [differential(lambda x: P(v, Ix(x)), x, b) for v in frame]
A1 = f_(PA)(x)
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)
fb_A1 = lambda C: differential(f__(C), x, pb, h=1e-2).sp(A1)

(
    derivative(fb_A1, x, alg, frame=frame, r_frame=r_frame, h=1e-2) +
    Pb_(fa_(A1))(x),
    fb(A1)
    )

(5.55e-15 + 0.83 𝐞₁ + -0.641 𝐞₂ + 0.15 𝐞₃ + 0.34 𝐞₁₂ + -0.173 𝐞₁₃ + 0.191 𝐞₂₃ + -1.6e-18 𝐞₁₂₃,
 0.83 𝐞₁ + -0.641 𝐞₂ + 0.15 𝐞₃ + 0.34 𝐞₁₂ + -0.173 𝐞₁₃ + 0.191 𝐞₂₃)

In [22]:
#5.20
(
    derivative(fb_A1, x, alg, frame=frame, r_frame=r_frame, h=1e-2),
    P(Pb_(fa_(A1))(x), I),
    P(fb(A1), I)
    )

(0.832 𝐞₁ + -0.644 𝐞₂ + 0.0315 𝐞₃ + 0.341 𝐞₁₂ + -0.00817 𝐞₁₃ + -0.00656 𝐞₂₃ + -1.6e-18 𝐞₁₂₃,
 5.55e-15 + 1.1e-07 𝐞₁ + -1.74e-07 𝐞₂ + 6.27e-09 𝐞₃ + 9.21e-09 𝐞₁₂ + -2.21e-10 𝐞₁₃ + -1.77e-10 𝐞₂₃,
 0.832 𝐞₁ + -0.644 𝐞₂ + 0.0315 𝐞₃ + 0.341 𝐞₁₂ + -0.00817 𝐞₁₃ + -0.00656 𝐞₂₃)

In [23]:
#5.21
C.sp(fb(A1)), fb_A1(C)

(-0.164, -0.164)

In [24]:
#5.22 quadriple differential, so the truncations...
f_vec = lambda a: lambda x: differential(f, x, a, h=1e-3)
fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
fb = lambda b: differential(fa_(A1), x, b, h=1e-3)
P(curl(fb, x, alg, Ix=Ix, h=1e-2), I)

5.55e-20 𝐞₃ + -7.02e-05 𝐞₁₂ + 1.68e-06 𝐞₁₃ + 1.35e-06 𝐞₂₃

In [25]:
# Even though these two are the same
f__(PA(x))(x), f__(A(x))(x)

(0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃ + 4.06e-18 𝐞₁₂₃,
 0.603 + 0.139 𝐞₁ + 0.945 𝐞₂ + -1.4 𝐞₃ + -0.646 𝐞₁₂ + 1.01 𝐞₁₃ + 0.376 𝐞₂₃ + 4.06e-18 𝐞₁₂₃)

In [26]:
#5.23 The non-tangent part of A contributes to the exterior derivative

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, A(x), alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, PA(x), alg, Ix, h=1e-2)

(0.0149 𝐞₁ + -0.0664 𝐞₂ + 0.113 𝐞₃ + -0.158 𝐞₁₂ + 0.247 𝐞₁₃ + 0.0919 𝐞₂₃ + -7.89e-16 𝐞₁₂₃,
 -8.87e-07 𝐞₁ + -2.56e-05 𝐞₂ + 1.03e-06 𝐞₃ + -1.82e-19 𝐞₁₂ + 2.85e-19 𝐞₁₃ + 1.06e-19 𝐞₂₃ + -7.88e-16 𝐞₁₂₃)

In [27]:
#5.23 How? The vector part contributes nothing

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, a(x), alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, pa(x), alg, Ix, h=1e-2)

(6.52e-19 𝐞₁₂ + 2.52e-20 𝐞₁₃ + 1.8e-19 𝐞₂₃,
 -1.82e-19 𝐞₁₂ + 2.85e-19 𝐞₁₃ + 1.06e-19 𝐞₂₃)

In [28]:
#5.23 The bivector part contributes
blade = random_r_blade(2, alg)
pblade = P(blade, I)

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, blade, alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, pblade, alg, Ix, h=1e-2)

(-0.0399 𝐞₁ + -0.0313 𝐞₂ + 0.0259 𝐞₃ + 1.49e-15 𝐞₁₂₃,
 1.68e-06 𝐞₁ + 4.86e-05 𝐞₂ + -1.96e-06 𝐞₃ + 1.49e-15 𝐞₁₂₃)

In [29]:
T = lambda x, A: f__(A)(x)
list((inner(blade, r), T(x, inner(blade, r)), o(lambda x: T(x, inner(blade, r)))) for r, o in derivative_gen(x, alg, nIx, h=1e-3))


[(-0.399 𝐞₁ + 0.0031 𝐞₂ + 0.129 𝐞₃,
  -0.83 𝐞₁ + -0.485 𝐞₂ + 0.277 𝐞₃ + -1.27e-21 𝐞₁₂₃,
  -0.202 𝐞₁ + 0.0322 𝐞₂ + -0.271 𝐞₃ + -4.76e-18 𝐞₁₂₃),
 (0.00703 𝐞₁ + -0.405 𝐞₂ + -0.365 𝐞₃,
  0.715 𝐞₁ + -0.168 𝐞₂ + 0.678 𝐞₃ + -2.75e-18 𝐞₁₂₃,
  0.162 𝐞₁ + -0.0636 𝐞₂ + 0.296 𝐞₃ + -1.38e-15 𝐞₁₂₃)]

In [30]:
T = lambda x, A: f__(A)(x)
list(o(lambda x: T(x, inner(pblade, r))) for r, o in derivative_gen(x, alg, nIx, h=1e-3))


[-0.212 𝐞₁ + 0.0345 𝐞₂ + -0.28 𝐞₃ + -4.74e-18 𝐞₁₂₃,
 0.212 𝐞₁ + -0.0345 𝐞₂ + 0.28 𝐞₃ + -1.38e-15 𝐞₁₂₃]

In [31]:
#5.24
d(lambda x, A: P(A, Ix(x)), x, PA(x), alg, Ix)

1.11e-10 + -4.92e-11 𝐞₁ + 2.09e-11 𝐞₂ + -6.51e-12 𝐞₃ + -5.87e-20 𝐞₁₂ + 1.41e-21 𝐞₁₃ + 1.13e-21 𝐞₂₃

In [32]:
#5.25
fb = lambda C: differential(fa_(C), x, pb, h=1e-3)
Pb_ = lambda A: lambda x: differential(lambda a: P(A(x), Ix(a)), x, pb, h=1e-3)
Pb_1 = lambda A: lambda x: differential(lambda a: P(A(x), f_(Ix)(a)), x, pb, h=1e-3)
P1C = P(C, I1)
C_ = lambda x: C - P1C
(
    fb(C),
    Pb_(fa_(C))(x)+ P(fb(C), I),
    fb(P1C) + fa_(Pb_1(C_)(x))(x)
    )

(0.0374 𝐞₁ + -0.275 𝐞₂ + 0.00838 𝐞₃ + -0.853 𝐞₁₂ + -0.0355 𝐞₁₃ + 0.0833 𝐞₂₃,
 0.0374 𝐞₁ + -0.275 𝐞₂ + 0.00838 𝐞₃ + -0.853 𝐞₁₂ + -0.0355 𝐞₁₃ + 0.0833 𝐞₂₃,
 0.0375 𝐞₁ + -0.275 𝐞₂ + 0.00838 𝐞₃ + -0.853 𝐞₁₂ + -0.0355 𝐞₁₃ + 0.0833 𝐞₂₃)

In [33]:
#5.25b
fb = lambda b: differential(fa_(C), x, b, h=1e-3)
Pb = lambda b: differential(lambda a: P(fa_(C)(x), Ix(a)), x, b, h=1e-3)
(
    curl(fb, x, alg, Ix=Ix, h=1e-3), 
    curl(Pb, x, alg, Ix=Ix, h=1e-3), 
    P(curl(fb, x, alg, Ix=Ix, h=1e-3), I)
    )


(0.000165 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00924 𝐞₂₃ + 0.268 𝐞₁₂₃,
 6.94e-12 𝐞₁ + -9.7e-12 𝐞₂ + 3.66e-13 𝐞₃ + 0.00017 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00924 𝐞₂₃ + 0.269 𝐞₁₂₃,
 8.88e-19 𝐞₂ + -3.79e-06 𝐞₁₂ + 9.1e-08 𝐞₁₃ + 7.31e-08 𝐞₂₃)

In [34]:
#5.25c
fb = lambda b: differential(fa_(P1C), x, b, h=1e-3)
Pb_1 = lambda b: differential(lambda a: P(C_(x), f_(Ix)(a)), x, b, h=1e-3)
fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
(
    curl(fb, x, alg, Ix=Ix, h=1e-3), 
    fa_(sum(r^o(Pb_1) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)))(x),
    )


(0.000203 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00923 𝐞₂₃ + 0.268 𝐞₁₂₃,
 -3.39e-26 𝐞₁ + -7.35e-23 𝐞₂ + 1.76e-24 𝐞₃ + -0.101 𝐞₁₂ + 0.00243 𝐞₁₃ + 0.00195 𝐞₂₃)

In [35]:
#5.26
S = lambda A: curl(lambda x: P(A, Ix(x)), x, alg, Ix=Ix, h=1e-3)
Pb = lambda b: differential(lambda a: P(fa_(C)(x), Ix(a)), x, b, h=1e-3)
fb = lambda C: lambda b: differential(fa_(C), x, b, h=1e-3)
(
    S(fa_(C)(x)), 
    curl(lambda a: P(fa_(C)(x), Ix(a)), x, alg, Ix=Ix, h=1e-3), 
    curl(Pb, x, alg, Ix=Ix, h=1e-3), 
    curl(fb(P1C), x, alg, Ix=Ix, h=1e-3), 
    curl(fb(C), x, alg, Ix=Ix, h=1e-3),
    )

(-6.94e-15 𝐞₁ + -1.94e-14 𝐞₂ + 3.32e-16 𝐞₃ + 0.00017 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00919 𝐞₂₃ + 0.268 𝐞₁₂₃,
 -6.94e-15 𝐞₁ + -1.94e-14 𝐞₂ + 3.32e-16 𝐞₃ + 0.00017 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00919 𝐞₂₃ + 0.268 𝐞₁₂₃,
 6.94e-12 𝐞₁ + -9.7e-12 𝐞₂ + 3.66e-13 𝐞₃ + 0.00017 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00924 𝐞₂₃ + 0.269 𝐞₁₂₃,
 0.000203 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00923 𝐞₂₃ + 0.268 𝐞₁₂₃,
 0.000165 𝐞₁₂ + 0.0145 𝐞₁₃ + -0.00924 𝐞₂₃ + 0.268 𝐞₁₂₃)

In [36]:
#5.27
C - P1C, curl(fb(C - P1C), x, alg, Ix=Ix, h=1e-3)

(-0.101 𝐞₁ + 0.273 𝐞₂ + 0.174 𝐞₃ + 0.313 𝐞₁₂ + 0.136 𝐞₁₃ + 0.173 𝐞₂₃ + 0.723 𝐞₁₂₃,
 -3.71e-05 𝐞₁₂ + -1.48e-05 𝐞₁₃ + -1.15e-05 𝐞₂₃ + 6.92e-05 𝐞₁₂₃)

In [37]:
C, fa_(C)(x), f__(C)(x), fa(f__(C))(x)

(0.0657 + 0.00233 𝐞₁ + 0.295 𝐞₂ + 0.2 𝐞₃ + 0.0943 𝐞₁₂ + 0.479 𝐞₁₃ + 0.3 𝐞₂₃ + 0.723 𝐞₁₂₃,
 0.0657 + 0.226 𝐞₁ + -0.222 𝐞₂ + 0.00968 𝐞₃ + -2.57 𝐞₁₂ + 0.0616 𝐞₁₃ + 0.0495 𝐞₂₃,
 0.0657 + -0.51 𝐞₁ + 0.13 𝐞₂ + -0.501 𝐞₃ + 0.238 𝐞₁₂ + -0.373 𝐞₁₃ + -0.139 𝐞₂₃ + 2.02e-18 𝐞₁₂₃,
 0.0657 + -0.558 𝐞₁ + 1.83 𝐞₂ + -0.0547 𝐞₃ + 2.79 𝐞₁₂ + -0.067 𝐞₁₃ + -0.0538 𝐞₂₃)

In [38]:
PC = P(C, I)
PC1 = P(C, I1)
PC1, fa_(PC1)(x), f__(PC)(x), fa(f__(PC))(x)

(0.0657 + 0.104 𝐞₁ + 0.0222 𝐞₂ + 0.0257 𝐞₃ + -0.219 𝐞₁₂ + 0.343 𝐞₁₃ + 0.128 𝐞₂₃,
 0.0657 + 0.226 𝐞₁ + -0.222 𝐞₂ + 0.00968 𝐞₃ + -2.57 𝐞₁₂ + 0.0616 𝐞₁₃ + 0.0495 𝐞₂₃,
 0.0657 + -0.51 𝐞₁ + 0.13 𝐞₂ + -0.501 𝐞₃ + 0.238 𝐞₁₂ + -0.373 𝐞₁₃ + -0.139 𝐞₂₃ + 2.02e-18 𝐞₁₂₃,
 0.0657 + -0.558 𝐞₁ + 1.83 𝐞₂ + -0.0547 𝐞₃ + 2.79 𝐞₁₂ + -0.067 𝐞₁₃ + -0.0538 𝐞₂₃)